In [5]:
#!pip install gensim
from nltk.translate.bleu_score import corpus_bleu

In [33]:
## import data processing/cleaning , data modeling libraries
#!pip install boto3

import pandas as pd
import os
import sys
import re as re
import datetime as datetime
import numpy as np
import collections
import boto3
import io
import string

#from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from nltk.tag.perceptron import PerceptronTagger

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import silhouette_score
    
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from gensim.models import doc2vec
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models import fasttext

import warnings
warnings.filterwarnings('ignore')

source="/home/cj-pythonrecsys/adviewRecomm/version3/temp/"
sparkfiles = "/home/cj-pythonrecsys/spark-modules/clust_vtitleTrans/"
destination=source

lang=["en"] # video language got from video title using python Library
clustSize = 0 # no of clusters - initialization
t0 = datetime.datetime.now()
print(source)

/home/cj-pythonrecsys/adviewRecomm/version3/temp/


In [34]:
## outData41 - video data coming from postgres DB
#  outData41.to_csv(source+"youtubeModelData_stats1.csv",index=False)

outData43 = pd.read_csv(source+"clipVdChanelData_model_in.csv",encoding="utf-8")
print(outData43.shape)
print(outData43.head(n=2)) # 39:
outData43 = outData43.drop_duplicates()
outData43 = outData43.dropna(subset=["v_title_trans"])
print(outData43.shape)

(274299, 19)
          v_id                                            v_title  \
0  YH6RzRnMVHE  Marigold Paper Flowers Designs | Easy Paper Cu...   
1  _EOzjBv-_RI  Fun Comedy Videos | Telugu Comedy Videos | Fun...   

                                       v_title_trans ch_lang_raw  v_seq_id  \
0  Marigold Paper Flowers Designs | Easy Paper Cu...       hindi   1238508   
1  Fun Comedy Videos | Telugu Comedy Videos | Fun...       hindi   1238515   

   v_duration                                            v_descr  \
0          94  Marigold Paper Flowers Designs | Easy Paper Cu...   
1         108  Fun Comedy Videos | Telugu Comedy Videos | Fun...   

                                        v_thumbnail  \
0  https://i.ytimg.com/vi/YH6RzRnMVHE/mqdefault.jpg   
1  https://i.ytimg.com/vi/_EOzjBv-_RI/mqdefault.jpg   

                                            ch_descr  ch_id       ch_name  \
0  Trying to define a city's atmosphere and energ...   1264  Madras Meter   
1  Trying to define

In [35]:
## NLP PRE-PROCESSING

#text=" the JanaSena Party Formation Day celebrations || LIVE funny celebration|| Pawan Kalyan || Guntur"
text="Chak De India | Full Title Song | Shah Rukh Khan | Sukhvinder Singh | Salim | Marianne D'Cruz"

def nltk_clean_sent(line):
    if len(line)>0:
        ## remove the punctuation/emoticons/digits/multispaces with single from the line
        ## dont make lowercase before the pos tagging
        line_lower = line.strip()
        line_punct = re.sub('['+string.punctuation+']',' ',line_lower)
        line_emots = re.sub(r'[\u200b-\u2fff]+',' ',line_punct)
        line_digis = re.sub(r'[0-9]+',' ',line_emots)
        line_spaces = re.sub(r'[\s]+',' ',line_digis)
        line = line_spaces
    return line

def nltk_extract_postags(line):
    cleaned_str = ''
    tokens = nltk.word_tokenize(line)
    tokens_pos = PerceptronTagger().tag(tokens)
    #print(tokens_pos)
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    
    for tag_word in tokens_pos:
        if tag_word[1] in nltk_tags:
            if len(tag_word[0])>2:
                cleaned_str += tag_word[0]+' '
    return cleaned_str.strip().lower()

def nltk_apply_lemma(line):
    tokens_lemmas = [WordNetLemmatizer().lemmatize(word) for word in line.split()]
    ## stemming
    tokens_stops = [word for word in tokens_lemmas if word not in stopwords.words('english')]
    tokens_stops = [word for word in tokens_stops if len(word.strip())>2]
    tokens_stops = list(set(tokens_stops))
    return tokens_stops

def nltk_extract_tags(line):
        ## tokenize the sentence/get tokens that contains only letters
        line_clean = nltk_clean_sent(line)
        ## apply postags to the words and get only couple of tags and word length >2
        tokens_pos = nltk_extract_postags(line_clean)
        ## apply lemmatize/stemming and remove stopwords
        token_lemma = nltk_apply_lemma(tokens_pos)
        return token_lemma
    
nltk_extract_tags(text)

['title',
 'khan',
 'singh',
 'song',
 'shah',
 'sukhvinder',
 'chak',
 'rukh',
 'salim',
 'marianne',
 'cruz',
 'full',
 'india']

In [36]:
## FASTEXT
# We distribute pre-trained word vectors for 157 languages, trained on Common Crawl and Wikipedia using fastText. 
# These models were trained using CBOW with position-weights, in dimension 300,with character n-grams of length 5, a window of size 5 and 10 negatives
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('/home/fastextvecs/wiki.en.bin')
print(model)

FastText(vocab=2519370, size=300, alpha=0.025)


In [2]:
# from gensim.models.wrappers import FastText
# print(FastText())

FastText(vocab=0, size=100, alpha=0.025)


In [38]:
# Compute clean title
outData43['clean_title_words'] = outData43.v_title_trans.apply(nltk_extract_tags)
#print(df_en.head(n=2))
outData43['clean_title_text'] = outData43.clean_title_words.apply(lambda x:' '.join(x))
print(outData43.head(n=2))

          v_id                                            v_title  \
0  YH6RzRnMVHE  Marigold Paper Flowers Designs | Easy Paper Cu...   
1  _EOzjBv-_RI  Fun Comedy Videos | Telugu Comedy Videos | Fun...   

                                       v_title_trans ch_lang_raw  v_seq_id  \
0  Marigold Paper Flowers Designs | Easy Paper Cu...       hindi   1238508   
1  Fun Comedy Videos | Telugu Comedy Videos | Fun...       hindi   1238515   

   v_duration                                            v_descr  \
0          94  Marigold Paper Flowers Designs | Easy Paper Cu...   
1         108  Fun Comedy Videos | Telugu Comedy Videos | Fun...   

                                        v_thumbnail  \
0  https://i.ytimg.com/vi/YH6RzRnMVHE/mqdefault.jpg   
1  https://i.ytimg.com/vi/_EOzjBv-_RI/mqdefault.jpg   

                                            ch_descr  ch_id  \
0  Trying to define a city's atmosphere and energ...   1264   
1  Trying to define a city's atmosphere and energ...   1264 

In [39]:
# TF-IDF transformation
def getTfIdfMetrics(outData44,lang):
    outData44_en = outData44[outData44.ch_lang_raw==lang]
    t0 = datetime.datetime.now()
    tfidf_transform = TfidfVectorizer(tokenizer=nltk_extract_tags,min_df=6,max_df=0.95,stop_words='english',use_idf=True,ngram_range=(1,1))
    #terms = tfidf_transform.get_feature_names()
    #print(terms[:100])
    tfidf_vecto = tfidf_transform.fit_transform(outData44_en["clean_title_text"])
    tf_idf_dict = dict(zip(tfidf_transform.get_feature_names(),tfidf_transform.idf_))
    print(tfidf_vecto.shape)
    # got shape of (83806, 149542) - without translation
    t1 = datetime.datetime.now()
    print("END TIME after TFIDF =",t1)
    print("time taken until TFIDF=",(t1-t0))
    return tf_idf_dict

## PRINTING FIRST 100 FEATURES
#getTfIdfMetrics(outData44,'telugu')
#terms = tfidf_transform.get_feature_names()
#print(terms[:100])

In [40]:
import numpy as np
embed_size = 300

def sentences_train_w_tf_idf(strings):
	# Compute the embedding of a sentence to a 300 dimension vector
	# Mix tf-idf with FastText on each word to resume to a vector the sentence
    #print(len(tf_idf_dict))
    len_string = len(strings)
    words = np.zeros((len_string,embed_size))
    w = np.zeros(len_string)
    for i,word in enumerate(strings):
        try:
            if word in tf_idf_dict.keys():
                words[i]=model.wv[word]
                words[i]=words[i]*tf_idf_dict[word]
        except:
            words[i]=np.random.rand(embed_size)
            #print("not found in fasttext")
    finalvec = np.sum(words,0)/len(words)
    return finalvec

In [41]:
## MODEL FUNCTION DEFINITION
#feature_vecto = np.stack(d2vmodel.docvecs)
def getModelDataTfIdf(outData5,lang,xtrain):
    ## chosing no of clusters
    if len(outData5) < 1600:
        clustSize=3
    else:
        clustSize = int(np.round(np.divide(len(outData5),1600)))
    print("cluster lang=",lang)    
    print("clust data len=",len(outData5))
    print("cluster size=",clustSize)

    ## KMeans model
    from sklearn.cluster import KMeans
    #kmeans = KMeans(n_clusters=clustSize,init='k-means++',max_iter=80).fit(feature_vecto)
    kmeans = KMeans(n_clusters=clustSize).fit(xtrain)
    #kmeans = GaussianMixture(n_components=clustSize).fit(xtrain)
    preds1 = kmeans.labels_

    ### NO worries on this PART ###
    print(collections.Counter(preds1))
    b=pd.DataFrame(preds1)
    clustData1=pd.DataFrame(outData5)
    clustData1["clustNum"]=pd.Series(preds1).values

    #print(clustData1.head(n=1))
    #print(clustData1.shape)
    #clustData1.to_csv(destination+"/clustData_"+lang+".csv",index=False)
    ##clustData.groupby(["chCategory","clustNo","v_title"]).size().sort_values(ascending=False)
    #print(clustData1.columns)
    return clustData1

In [42]:
## LANGUAGE LEVEL CLUSTERING
startT = datetime.datetime.now()
langs = ['bengali','telugu','tamil','english','hindi']
#langs = ['english']
for lang in langs:
        outData5 = outData43[outData43.ch_lang_raw==lang]
        ## Dataframe Length Check
        if(len(outData5)>0):
            print(outData5.shape)
            
            ## feature engineering
            tf_idf_dict = getTfIdfMetrics(outData5,lang)
            print(len(tf_idf_dict))
            outData5['vectors_word2vec_tf_idf'] = outData5.clean_title_words.apply(sentences_train_w_tf_idf)
            xtrain = outData5.vectors_word2vec_tf_idf.apply(lambda x: pd.Series(list(x)))
            print("xtrain shape=",xtrain.shape)
            #print(xtrain.head(n=2))
            xtrain= xtrain.round(2)
            xtrain.fillna(xtrain.mean(),inplace=True)
            #print(xtrain.info())
            #xtrain=xtrain.replace([np.inf, -np.inf],0)
            print("outData5 shape=",outData5.shape)
#             #feature_vecto = getWord2VecMetrics(outData5,lang)
#             ## Model Data Function Call
            clustData2 = getModelDataTfIdf(outData5,lang,xtrain)
            print("final data=",clustData2.shape)
            print(clustData2.columns)
            clustData2.to_csv(destination+"/clustData_"+lang+".csv",index=False)
endT = datetime.datetime.now()
print("TIME taken for Modeling=",(endT-startT))
## 50 min for modelling - 274K

(32639, 21)
(32639, 4788)
END TIME after TFIDF = 2018-09-05 16:59:36.294668
time taken until TFIDF= 0:00:57.469035
4788
xtrain shape= (32639, 300)
outData5 shape= (32639, 22)
cluster lang= bengali
clust data len= 32639
cluster size= 20
Counter({1: 4391, 10: 3436, 3: 3022, 15: 2505, 7: 2448, 18: 2014, 5: 1995, 14: 1863, 6: 1617, 4: 1616, 8: 1481, 0: 1443, 16: 1357, 9: 960, 11: 960, 12: 741, 2: 617, 19: 78, 13: 58, 17: 37})
final data= (32639, 23)
Index(['v_id', 'v_title', 'v_title_trans', 'ch_lang_raw', 'v_seq_id',
       'v_duration', 'v_descr', 'v_thumbnail', 'ch_descr', 'ch_id', 'ch_name',
       'ch_thumbnail', 'v_categ_id', 'ch_active', 'ch_is_clickable',
       'chSource', 'v_isdownloaded', 'v_alias', 'ch_alias',
       'clean_title_words', 'clean_title_text', 'vectors_word2vec_tf_idf',
       'clustNum'],
      dtype='object')
(25982, 21)
(25982, 3681)
END TIME after TFIDF = 2018-09-05 17:05:08.685585
time taken until TFIDF= 0:00:47.354092
3681
xtrain shape= (25982, 300)
outData5

In [43]:
# print(clustData2.shape)
# clustData21 = clustData2.drop_duplicates(subset=['v_id'])
# print(clustData21.shape)
# clustData21 = clustData21[clustData21.clustNum==2]
# print(clustData21.shape)
# print(clustData21.head(n=10))

In [44]:
## sentence vector representation

# def sent_vectorizer(sent, model):
#     sent_vec = np.zeros(400)
#     numw = 0
#     for w in sent:
#         try:
#             sent_vec = np.add(sent_vec, model[w])
#             numw+=1
#         except:
#             pass
#     return sent_vec / np.sqrt(sent_vec.dot(sent_vec))

In [45]:
#     from sklearn.cluster import k_means_
#     from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances

#     # Manually override euclidean
#     def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
#         #return pairwise_distances(X, Y, metric = 'cosine', n_jobs = 10)
#         return cosine_similarity(X, Y)
#     k_means_.euclidean_distances = euc_dist

#     kmeans = k_means_.KMeans(n_clusters = clustSize,random_state = 3425).fit(tfidf_vecto)